# Context-Aware Emotion-Based Content Recommendation System

## 1. Problem Definition & Objective

### Selected Project Track
Content Recommendation System

### Problem Statement
Traditional content recommendation systems rely heavily on explicit user preferences or long-term behavioral tracking. However, in many real-world scenarios, users seek content based on their current mental state, available attention span, and short-term intent rather than historical behavior.

This project aims to design a context-aware content recommendation system that dynamically adapts recommendations based on:
- User emotional state inferred from natural language input
- Cognitive time availability (short / medium / long)
- Session-level engagement modeling

### Objective
To build a lightweight, interpretable, and responsible AI system that:
- Requires no user history
- Works in real time
- Provides emotionally intelligent content recommendations

In [ ]:
from IPython.display import display

# Sample inputs:
# 1. I am feeling great! Show me something exciting and long
# 2. I am stressed with work and need a short relaxing break
# 3. Feeling low and need some comfort
# 4. I want something detailed and informative
# 5. I am overwhelmed and want something comforting

def highlight_emotion(val):
    colors = {
        "happy": "#d4edda",
        "neutral": "#fff3cd",
        "sad": "#f8d7da",
        "stressed": "#d1ecf1"
    }
    return f"background-color: {colors.get(val, 'white')}; font-weight: bold"

while True:
    user_text = input("Enter your current mood/context (type 'exit' to stop): ")

    if user_text.lower() == "exit":
        print("Demo ended.")
        break

    emotion, time_state, engagement, recs = recommend_content(user_text)
    score = context_match_score(recs, time_state)

    print("\nDetected Emotion            :", emotion)
    print("Cognitive Time Availability :", time_state)
    print("Context Match Score         :", score)
    print("\nTop Recommended Content:\n")

    display(
        recs[["title", "emotion_tag", "duration", "reason"]]
        .style
        .map(highlight_emotion, subset=["emotion_tag"])
        .set_properties(**{"text-align": "center"})
    )


## 2. Real-World Relevance & Motivation

In real-world applications such as learning platforms, wellness apps, and content discovery systems, users often consume content based on their current emotional and cognitive context.

### Example Scenarios
- A stressed user may prefer short and calming content
- A motivated user may prefer long and engaging material
- A neutral user may prefer informative content

### Motivation
By modeling implicit user context, this system:
- Improves recommendation relevance
- Reduces cognitive overload
- Avoids long-term user profiling
- Encourages responsible personalization

## 3. Data Understanding & Preparation

### Dataset Source
Synthetic dataset generated programmatically.

### Why Synthetic Data?
- No personal or sensitive data used
- Full control over balance and fairness
- Suitable for academic evaluation

### Dataset Description
- 600 content items
- Emotion tags: happy, neutral, sad, stressed
- Duration categories: short, medium, long

In [1]:
import pandas as pd
import random

random.seed(42)

emotions = ["happy", "neutral", "sad", "stressed"]
durations = ["short", "medium", "long"]

data = []
content_id = 1

for emotion in emotions:
    for i in range(150):
        title = f"{emotion.capitalize()} Content {i+1}"
        duration = random.choice(durations)
        data.append([content_id, title, emotion, duration])
        content_id += 1

df = pd.DataFrame(
    data,
    columns=["content_id", "title", "emotion_tag", "duration"]
)

df.head()

,content_id,title,emotion_tag,duration
0,1,Happy Content 1,happy,long
1,2,Happy Content 2,happy,short
2,3,Happy Content 3,happy,short
3,4,Happy Content 4,happy,long
4,5,Happy Content 5,happy,medium


## 4. Model / System Design

### AI Techniques Used
- Rule-based Natural Language Processing
- Sentiment analysis (TextBlob)
- Hybrid emotion scoring
- Context-aware weighted recommendation
- Heuristic re-ranking

### System Architecture
1. User provides free-text input
2. Emotional context inferred using lexical cues and sentiment polarity
3. Cognitive time availability inferred from language patterns
4. Content scored using emotion alignment and attention suitability
5. Top recommended content returned with explanations

### Design Justification
This hybrid approach ensures:
- Interpretability
- Low computational cost
- Real-time adaptability

## 5. Core Implementation

In [2]:
from textblob import TextBlob

happy_keywords = [
    "happy","great","exciting","fun","awesome","amazing","motivated",
    "energetic","joy","cheerful","thrilled","fantastic","uplifting"
]

neutral_keywords = [
    "informative","overview","guide","tutorial","basic","simple",
    "quick","short","summary","explanation","general"
]

sad_keywords = [
    "sad","low","down","tired","unhappy","lonely","exhausted",
    "gloomy","disappointed","blue"
]

stressed_keywords = [
    "stressed","overwhelmed","anxious","pressure","tense",
    "worried","panic","busy","rushed"
]

### Advanced Emotion Detection

Emotion detection uses a hybrid scoring approach combining keyword frequency and sentiment polarity to improve robustness while remaining interpretable.

In [3]:
def detect_emotion_advanced(text):
    text = text.lower()
    polarity = TextBlob(text).sentiment.polarity

    emotion_scores = {
        "happy": 0,
        "neutral": 0,
        "sad": 0,
        "stressed": 0
    }

    for w in happy_keywords:
        if w in text:
            emotion_scores["happy"] += 1

    for w in sad_keywords:
        if w in text:
            emotion_scores["sad"] += 1

    for w in stressed_keywords:
        if w in text:
            emotion_scores["stressed"] += 1

    for w in neutral_keywords:
        if w in text:
            emotion_scores["neutral"] += 0.5

    if polarity > 0.4:
        emotion_scores["happy"] += 1
    elif polarity < -0.3:
        emotion_scores["sad"] += 1

    return max(emotion_scores, key=emotion_scores.get)

### Context-Aware Content Recommendation

Recommended content is generated using weighted emotion alignment, cognitive time suitability, and session engagement sensitivity.

In [4]:
def recommend_content(user_text, engagement=0.5, top_n=5):
    emotion = detect_emotion_advanced(user_text)

    if any(w in user_text.lower() for w in ["quick","short","tip"]):
        time_state = "short"
    elif any(w in user_text.lower() for w in ["long","detailed","deep"]):
        time_state = "long"
    else:
        time_state = "medium"

    emotion_map = {
        "sad": {"happy": 1.0, "neutral": 0.7},
        "stressed": {"happy": 1.0, "neutral": 0.8},
        "neutral": {"neutral": 1.0},
        "happy": {"happy": 1.0}
    }

    recommendations = []

    for _, row in df.iterrows():
        score = 0
        reasons = []

        if row["emotion_tag"] in emotion_map[emotion]:
            score += emotion_map[emotion][row["emotion_tag"]]
            reasons.append("emotion alignment")

        if row["duration"] == time_state:
            score += 0.6
            reasons.append("attention match")

        score *= (0.8 + engagement)

        recommendations.append(
            (score, row["title"], row["emotion_tag"], row["duration"], ", ".join(reasons))
        )

    recommendations = sorted(recommendations, reverse=True)

    return emotion, time_state, engagement, pd.DataFrame(
        recommendations[:top_n],
        columns=["score","title","emotion_tag","duration","reason"]
    )

## 6. Evaluation & Analysis

### Evaluation Approach
- Qualitative testing with diverse user inputs
- Transparent, human-interpretable explanations
- Lightweight quantitative scoring

### Limitations
- Rule-based emotion detection
- Synthetic dataset

In [5]:
def context_match_score(rec_df, expected_duration):
    duration_match = (rec_df["duration"] == expected_duration).mean()
    emotion_diversity = rec_df["emotion_tag"].nunique() / 4
    return round(0.7 * duration_match + 0.3 * emotion_diversity, 2)

In [16]:
from IPython.display import display

# Multiple demo inputs
demo_inputs = [
    "I am feeling great! Show me something exciting and long",
    "I just want a quick tip for cooking",
    "Feeling low and need some comfort",
    "I am stressed with work and need a short relaxing break",
    "I want to learn something detailed and informative today",
    "I feel tired and need something soothing",
    "Show me something fun and uplifting",
    "I feel overwhelmed and anxious about work"
]

# Color mapping for emotions
def highlight_emotion(val):
    colors = {
        "happy": "#d4edda",     # green
        "neutral": "#fff3cd",   # yellow
        "sad": "#f8d7da",       # red
        "stressed": "#d1ecf1"   # blue
    }
    return f"background-color: {colors.get(val, 'white')}; font-weight: bold"

# Run demo
for idx, text in enumerate(demo_inputs, 1):
    print("=" * 90)
    print(f"Demo Input {idx}: {text}")
    
    emotion, time_state, engagement, recs = recommend_content(text)
    score = context_match_score(recs, time_state)
    
    print(f"Detected Emotion            : {emotion}")
    print(f"Cognitive Time Availability : {time_state}")
    print(f"Context Match Score         : {score}")
    print("\nTop Recommended Content:\n")
    
    display(
        recs[["title", "emotion_tag", "duration", "reason"]]
        .style
        .map(highlight_emotion, subset=["emotion_tag"])   # ✅ FIXED HERE
        .set_properties(**{"text-align": "center"})
    )

Demo Input 1: I am feeling great! Show me something exciting and long
Detected Emotion            : happy
Cognitive Time Availability : long
Context Match Score         : 0.77

Top Recommended Content:



,title,emotion_tag,duration,reason
0,Happy Content 95,happy,long,"emotion alignment, attention match"
1,Happy Content 94,happy,long,"emotion alignment, attention match"
2,Happy Content 92,happy,long,"emotion alignment, attention match"
3,Happy Content 91,happy,long,"emotion alignment, attention match"
4,Happy Content 9,happy,long,"emotion alignment, attention match"


Demo Input 2: I just want a quick tip for cooking
Detected Emotion            : neutral
Cognitive Time Availability : short
Context Match Score         : 0.77

Top Recommended Content:



,title,emotion_tag,duration,reason
0,Neutral Content 99,neutral,short,"emotion alignment, attention match"
1,Neutral Content 96,neutral,short,"emotion alignment, attention match"
2,Neutral Content 95,neutral,short,"emotion alignment, attention match"
3,Neutral Content 94,neutral,short,"emotion alignment, attention match"
4,Neutral Content 92,neutral,short,"emotion alignment, attention match"


Demo Input 3: Feeling low and need some comfort
Detected Emotion            : sad
Cognitive Time Availability : medium
Context Match Score         : 0.77

Top Recommended Content:



...

,title,emotion_tag,duration,reason
0,Happy Content 99,happy,medium,"emotion alignment, attention match"
1,Happy Content 98,happy,medium,"emotion alignment, attention match"
2,Happy Content 86,happy,medium,"emotion alignment, attention match"
3,Happy Content 83,happy,medium,"emotion alignment, attention match"
4,Happy Content 82,happy,medium,"emotion alignment, attention match"


Demo Input 7: Show me something fun and uplifting
Detected Emotion            : happy
Cognitive Time Availability : medium
Context Match Score         : 0.77

Top Recommended Content:



,title,emotion_tag,duration,reason
0,Happy Content 99,happy,medium,"emotion alignment, attention match"
1,Happy Content 98,happy,medium,"emotion alignment, attention match"
2,Happy Content 86,happy,medium,"emotion alignment, attention match"
3,Happy Content 83,happy,medium,"emotion alignment, attention match"
4,Happy Content 82,happy,medium,"emotion alignment, attention match"


Demo Input 8: I feel overwhelmed and anxious about work
Detected Emotion            : stressed
Cognitive Time Availability : medium
Context Match Score         : 0.77

Top Recommended Content:



,title,emotion_tag,duration,reason
0,Happy Content 99,happy,medium,"emotion alignment, attention match"
1,Happy Content 98,happy,medium,"emotion alignment, attention match"
2,Happy Content 86,happy,medium,"emotion alignment, attention match"
3,Happy Content 83,happy,medium,"emotion alignment, attention match"
4,Happy Content 82,happy,medium,"emotion alignment, attention match"


## 7. Ethical Considerations & Responsible AI

- No personal or sensitive data used
- No long-term user tracking
- Transparent recommendation logic
- Balanced synthetic dataset to reduce bias
- User well-being prioritized over engagement

## 8. Conclusion & Future Scope

### Conclusion
This project demonstrates that emotionally intelligent content recommendation can be achieved using lightweight and interpretable AI techniques without relying on historical user behavior or invasive data collection.

### Future Scope
- Transformer-based emotion classification
- User feedback-driven learning
- Privacy-preserving multi-session modeling
- Domain-specific personalization